In [ ]:
# !pip install google-cloud
# !pip install google-cloud-vision
# !pip install --upgrade google-cloud-storage

In [ ]:
# !pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

In [68]:
import os
from datetime import datetime, timedelta
from google.cloud import bigquery

In [49]:
access_key_path = "/Users/weiyili/Desktop/Projects/spotify_twitter/airflow_twitter/key-file.json"

In [50]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = access_key_path

In [116]:
PROJECT_ID = "graphic-segment-355021"
DATASET_ID = "spotify_data"
TABLE_NAME_1 = "albums"
TABLE_NAME_2 = "tweets"

TABLE_ID_1 = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME_1}"
TABLE_ID_2 = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME_2}"

In [51]:
bqclient = bigquery.Client().from_service_account_json(access_key_path)

In [127]:
start_time = datetime.now()

In [128]:
time_diff = datetime.now() - start_time
row_num = int((time_diff.total_seconds()/60/5)%15)

In [144]:
query_string = f"""
SELECT text
FROM {TABLE_ID_2}
ORDER BY release_date DESC
LIMIT 1
"""

In [130]:
df = (bqclient.query(query_string).result().to_dataframe(create_bqstorage_client=True))

In [132]:
rd = df.loc[0, "release_date"]
an = df.loc[0, "album_name"]
au = df.loc[0, "album_url"]

In [133]:
tweet_post = f"On {rd}, #MariahCarey released her album '{an}'. Check out on Spotify:{au}."

In [134]:
row_to_insert = [{"release_date": rd, "text": tweet_post}]
bqclient.insert_rows_json(TABLE_ID_2, row_to_insert)

[]

In [146]:
def make_tweet(start_time):
    # Authentication to grant access to BigQuery tables
    access_key_path = "/Users/weiyili/Desktop/Projects/spotify_twitter/airflow_twitter/key-file.json"
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = access_key_path
    
    # Set necesary information
    PROJECT_ID = "graphic-segment-355021"
    DATASET_ID = "spotify_data"
    TABLE_NAME_1 = "albums"
    TABLE_NAME_2 = "tweets"

    TABLE_ID_1 = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME_1}"
    TABLE_ID_2 = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME_2}"   
    
    # Initialize the client object to interact with BigQuery
    bqclient = bigquery.Client().from_service_account_json(access_key_path)
    
    # Determine which certain row to retrieve
    time_diff = datetime.now() - start_time
    row_num = int((time_diff.total_seconds()/60/5)%15)
    
    # SQL query
    query_string = f"""
    SELECT *
    FROM {TABLE_ID_1}
    ORDER BY release_date
    LIMIT 1 OFFSET {row_num}
    """
    
    # Query job against BigQuery table and transform the results into a data frame
    df = (bqclient.query(query_string).result().to_dataframe(create_bqstorage_client=True))
    
    # Obtain certain fields and prepare the rows to be inserted into the table
    rd = df.loc[0, "release_date"]
    an = df.loc[0, "album_name"]
    au = df.loc[0, "album_url"]   
    tweet_post = f"On {rd}, #MariahCarey released her album '{an}'. Check out on Spotify:{au}."
    row_to_insert = [{"release_date": rd, "text": tweet_post}]
    
    # Perform insertion
    bqclient.insert_rows_json(TABLE_ID_2, row_to_insert)

In [147]:
make_tweet(start_time)